In [4]:
import numpy as np
import os
import glob
import math
import nibabel as nib
import cv2

from pycimg import CImg
from skimage import measure
from skimage import morphology
from scipy import ndimage
from scipy.spatial import ConvexHull


In [6]:
fname = './Kody_Pomiary/SEG_IMG_0001.nii.gz'

foo = nib.load(fname).get_fdata()

print(foo.shape)
CImg(foo).display();


(512, 512, 133)


: this = 0x5637f17f6210, size = (133,512,512,1) [133 Mio], data = (float32*)0x7f7c0ebfd010..0x7f7c170fd00f (non-shared) = [ 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 ], min = 0, max = 1, mean = 0.0874465, std = 0.282488, coords_min = (0,0,0,0), coords_max = (57,165,126,0).


In [8]:
labels,_ = ndimage.label(foo)

if np.sum(labels) > 0:
    props = measure.regionprops(labels)
    ids,counts = np.unique(labels,return_counts=True)
    
    for i,c,regionProp in zip(ids[1:],counts[1:],props):   #bez tła

        voxels = np.where(labels==i)
        voxels = np.asarray(voxels)
        voxels = np.swapaxes(voxels,0,1)
        chull = ConvexHull(voxels,qhull_options = 'QJ')

        dum = np.zeros(labels.shape,dtype=np.float32)
        dum[labels==i] = 1
        verts, faces, _, _ = measure.marching_cubes(dum, 0.0)
        surf = measure.mesh_surface_area(verts, faces)

        print(fname,i, \
                surf, \
                math.pow(3*c/4/math.pi,1/3)/math.pow(surf/4/math.pi,1/2), \
                regionProp.area, \
                regionProp.inertia_tensor_eigvals[0], \
                regionProp.inertia_tensor_eigvals[1], \
                regionProp.inertia_tensor_eigvals[2], \
                regionProp.major_axis_length, regionProp.minor_axis_length, \
                regionProp.bbox_area, \
                regionProp.equivalent_diameter, \
                chull.volume,chull.area)


./Kody_Pomiary/SEG_IMG_0001.nii.gz 1 270257.5 0.6133821183564276 3048834.0 12234.467381712919 8983.36055531602 4073.623865524852 414.0556009946501 90.69272512875291 11064960.0 179.90575334901135 5386177.000171818 199829.13440974746
